<h1>Select Dates for Monthly Reports via Selenium</h1>

<h1> OCR - Scrape PNG Screenshot Site Data </h1>

In [ ]:
# data scraping
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver import ActionChains

#data conditioning
import datetime as dt
import re

In [ ]:
 # add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# git repo folder
git_folder = 'd:/git'

# dictionary Directory
dictionary_dir = f'{git_folder}/dictionaries'

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports'

In [ ]:
# import configparser for env secrets
from configparser import ConfigParser

config = ConfigParser()
config.read(f'{git_folder}/config/env.ini')

from requests.structures import CaseInsensitiveDict

# Report Changes Configuration

In [ ]:
distribution_emails = [{"Name": "Reports",
                        "Address": "reports@example.co"}]

In [ ]:
report_days = [{"reportName":"*Monthly Reports - Software Distribution",
                "reportExportDay": "1"},
               {"reportName": "*Monthly Reports - Remote Activity",
                "reportExportDay": "Last"}]

## Create Chrome Driver

In [ ]:
chrome_driver = 'D:/data_science_utilities/web_drivers/chrome/chromedriver_win32/chromedriver.exe'
options = webdriver.ChromeOptions()
# options.add_argument('headless')
c_driver = webdriver.Chrome(executable_path=chrome_driver)

## Request Header Info (if needed to impersonate api browser calls)

## Create Firefox Driver

In [ ]:
# construct header
headers = CaseInsensitiveDict()
headers[
    'User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'

# construct req body
data = CaseInsensitiveDict()
data = ''

In [ ]:
baseUrl = 'https://concord.centrastage.net'
auth_url = f'https://auth.datto.com'
#home_url = f'{baseUrl}'
links = []

# Login to DattoRMM

In [ ]:
c_driver.get(auth_url)

In [ ]:
login_txtbox = c_driver.find_element("xpath", "//input[@id='form_username']")

In [ ]:
login_txtbox.send_keys((config['dattormm']['username']), Keys.ENTER)

In [ ]:
pass_txtbox = c_driver.find_element("xpath", "//input[@id='form_password']")
pass_txtbox.send_keys((config['dattormm']['password']), Keys.ENTER)

In [ ]:
mfa_txtbox = c_driver.find_element("id", "authy_token")
mfa_token = input()
mfa_txtbox.send_keys(mfa_token, Keys.ENTER)  # number entered manually for now /TODO: automate MFA

# Navigate to reports

In [ ]:
report_tab_selected = "completed"

In [ ]:
reports_url = f"{baseUrl}/csm/report/reports?mode={report_tab_selected}"
c_driver.get(reports_url)

time.sleep(5)

In [ ]:
report_a_list = c_driver.find_elements("xpath", "//a[contains(text(),'Monthly Report')]")

In [ ]:
datto_role_dropbox = c_driver.find_element("xpath", '//select[@name="roles"]')

In [ ]:
select = Select(datto_role_dropbox)

In [ ]:
select.select_by_visible_text('administrator')

In [ ]:
reports_info_list = True
#
# try:
#     report_tab_selected = "completed"
#     reports_url = f"{baseUrl}/csm/report/reports?mode={report_tab_selected}"
#     c_driver.get(reports_url)
#     report_a_list = c_driver.find_elements("xpath", "//a[contains(text(),'Monthly Report')]")
#
#     for a in report_a_list:
#         report_info = {'title': a.text, 'href': a.get_attribute('href')}
#         reports_info_list.append(report_info)
# except:
#     pass
#
# # reports_info_list[0]
#
# counter = 0

while reports_info_list:

    reports_info_list = []

    try:
        report_tab_selected = "completed"
        reports_url = f"{baseUrl}/csm/report/reports?mode={report_tab_selected}"
        c_driver.get(reports_url)
        report_a_list = c_driver.find_elements("xpath", "//a[contains(text(),'Monthly Report')]")

        for a in report_a_list:
            report_info = {'title': a.text, 'href': a.get_attribute('href')}
            reports_info_list.append(report_info)
    except:
        # break
        pass

    report_config = reports_info_list[0]

    # Click on Report and Inside Report
    c_driver.get(report_config['href'])

    # Select Range as Current Month
    try:
        current_month_radio = c_driver.find_element("xpath", "//input[@id='deviceActivityExportCurrentMonth']")
        current_month_radio.send_keys(Keys.SPACE)
    except:
        # continue
        pass

    # Change Schedule
    change_schedule = c_driver.find_element("xpath", "//input[@value='Click to change ...']")
    change_schedule.click()

    ## Inside Report Schedule find Table Box
    schedule_table = c_driver.find_element("xpath", "//table[@id='schedule']")

    ## Select 'Monthly'
    schedule_monthly = schedule_table.find_element("xpath", ".//input[@type='radio' and @id='monthly' and @name='when']")
    schedule_monthly.send_keys(Keys.SPACE)

    ## Select 'Month Checkboxes'

    ### Find Months Checkboxes > Select if False
    month_checkboxes = schedule_table.find_elements("xpath", ".//input[contains(@id,'monthlymonth')]")
    for month_checkbox in month_checkboxes:
        # print(month_checkbox.is_selected())
        if month_checkbox.is_selected() is False:
            month_checkbox.send_keys(Keys.SPACE)

    ## Find and compare report day
    for report in report_days:
        if (report['reportName']) in report_config['title']:
            day_of_report = (report['reportExportDay'])

    ### Find Days Checkboxes and Check if isSelected() == False
    day_checkboxes = schedule_table.find_elements("xpath", ".//input[contains(@id,'monthlyday')]")
    for day_checkbox in day_checkboxes:
        # id = day_checkbox.get_attribute('id')
        day = (re.match(r'monthlydayNum(\w+)', day_checkbox.get_attribute('id'))).group(1)
        if day == day_of_report:
            day_elem = schedule_table.find_element("xpath",f".//input[@id='monthlydayNum{day}']")
            if day_elem.is_selected() is False:
                day_elem.send_keys(Keys.SPACE)

    ok_button = schedule_table.find_element("xpath",".//input[@value='OK' and @type='button']").send_keys(Keys.SPACE)
    recipients_table = c_driver.find_element("xpath","//div[@id='reportRecipientDiv']")


    tbody_recipient_list = recipients_table.find_elements("xpath",".//td")

    box_objects = ["name","address","action"]

    i = 0
    box_info_list = []
    box_info = {}
    reports_email_present = False
    for element in tbody_recipient_list:

        if i == 2:
            box_info.update({box_objects[i]: element})
        else:
            box_info.update({box_objects[i]: element.text})
            if element.text == "reports@example.co":
                print(element.text)
                reports_email_present = True

        i = i + 1
        if i >= len(box_objects):
            i = 0
            box_info_list.append(box_info)
            box_info = {}
            print("*"*50)

    if reports_email_present is False:
        raise Exception("Reports Email Not Configured")


    save_button = c_driver.find_element("xpath","//input[@type='submit' and @value='Save']")
    save_button.click()